## 完整训练套路

In [ ]:
# 准备数据集
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
import torch 

train_data = torchvision.datasets.CIFAR10("./_data", train=True, 
            transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10("./_data", train=False, 
            transform=torchvision.transforms.ToTensor())

# length 长度 
train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"训练集数据的长度为：{train_data_size}")
print(f"测试集数据的长度为：{test_data_size}")

# 利用 DataLoader 加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [ ]:
# 搭建神经网络
class Foo(nn.Module):

    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)        
        )

    def forward(self, x):
        x = self.model(x)
        return x 

In [ ]:
# 验证网络输入输出准确性
foo = Foo()
input_t = torch.ones((64, 3, 32, 32))
output = foo(input_t)
print(f"输出的维度为：{output.shape}")

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# 创建模型
foo = Foo()
# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.SGD(foo.parameters(), lr=1e-2)

# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10


# 添加 tensorboard 
writer = SummaryWriter("./_log/train_proc")

for i in range(epoch):
    print(f"-------------第{i}轮训练开始-------------")

    # 训练步骤开始
    for data in train_dataloader:
        imgs, labels = data
        outputs = foo(imgs)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        # 记录训练信息
        if total_train_step % 100 == 0:
            print(f"训练次数：{total_train_step}，loss：{loss}")
            writer.add_scalar("train_loss", loss, total_train_step)

    # 测试步骤开始
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, labels = data
            outputs = foo(imgs)
            preds = outputs.argmax(1)
            total_accuracy += (preds == labels).sum()
            loss = loss_fn(outputs, labels)
            total_test_loss += loss

    print(f"整体测试集上的loss：{total_test_loss}")
    print(f"整体测试集上的准确率：{total_accuracy/test_data_size}")
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1

    torch.save(foo, f"./_model/foo_{i}.pth")
writer.close()

## 利用 GPU 进行训练

方式：  
对以下对象调用 .cuda() 方法
1. 网络模型
2. 数据（输入，标注）
3. 损失函数


In [ ]:
foo = Foo()
# 模型调用 cuda
# 使用判断增加泛用性
if torch.cuda.is_available():
    foo = foo.cuda()

loss_fn = nn.CrossEntropyLoss()
# 损失调用 cuda
loss_fn = loss_fn.cuda()


optimizer = torch.optim.SGD(foo.parameters(), lr=1e-2)
total_train_step = 0
total_test_step = 0
epoch = 10


# 添加 tensorboard 
writer = SummaryWriter("./_log/train_proc")

for i in range(epoch):
    print(f"-------------第{i}轮训练开始-------------")

    # 训练步骤开始
    for data in train_dataloader:
        imgs, labels = data
        # 数据调用 cuda 
        imgs, labels = imgs.cuda(), labels.cuda()
        outputs = foo(imgs)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            print(f"训练次数：{total_train_step}，loss：{loss}")
            writer.add_scalar("train_loss", loss, total_train_step)

    # 测试步骤开始
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, labels = data
            imgs, labels = imgs.cuda(), labels.cuda()
            outputs = foo(imgs)
            preds = outputs.argmax(1)
            total_accuracy += (preds == labels).sum()
            loss = loss_fn(outputs, labels)
            total_test_loss += loss

    print(f"整体测试集上的loss：{total_test_loss}")
    print(f"整体测试集上的准确率：{total_accuracy/test_data_size}")
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1

    torch.save(foo, f"./_model/foo_{i}.pth")
writer.close()


## 第二种调用 GPU 方式

In [ ]:
# 定义训练设备
device = torch.device("cpu")

foo = Foo()
foo = foo.to(device)

loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)